In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)
tf.random.set_seed(2)

2024-02-06 19:36:51.258654: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 19:36:51.282338: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 19:36:51.282361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 19:36:51.283081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 19:36:51.287507: I tensorflow/core/platform/cpu_feature_guar

# creating classifier

# load classifier

In [2]:
classifier_model_path = r"/home/webphy/Desktop/dnn_processor/neural_network/handmade/saved_models/manual_test_4x/model.h5"
classifier = tf.keras.models.load_model(classifier_model_path)


2024-02-06 19:36:52.392836: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-06 19:36:52.416900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-06 19:36:52.417048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers:
            print(l2)
    else:
        print(l.name)


input_2
conv2d_3
activation_2
conv2d_4
activation_3
conv2d_5
flatten_1
dense_1


# Save model weights as numpy file

the name of the file indicates the 'type' of the layer, the layer 'name', and 'activation', separeted by '.'.\
eg: Conv2D.encoder_output.Linear

In [4]:
def save_layer_file(id, layer, path):
    layer_name = layer.name
    layer_type = None
    activation_type = "Linear"
    weights, bias = [[], []]
    
    layer_config = layer.get_config()

    if isinstance(layer, tf.keras.layers.Conv2D):
        layer_type = "Conv2D"
        weights, bias = layer.get_weights()
        activation_type = layer_config["activation"]

    elif isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer_type = "DepthwiseConv2D"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Dense):
        layer_type = "Dense"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Activation):
        layer_type = "Activation"
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Flatten):
        layer_type = "Flatten"

    elif isinstance(layer, tf.keras.layers.InputLayer):
        layer_type = "InputLayer"

    elif isinstance(layer, tf.keras.layers.Dropout):
        layer_type = "Dropout"

    else:
        print(f"layer type {type(layer)} not expected..")

    weight_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.weights.npy"
    bias_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.bias.npy"

    np.save(
        f"{path}/" + weight_path,
        weights,
    )
    np.save(
        f"{path}/" + bias_path,
        bias,
    )

    layerJson = {
        "layerName": layer_name,
        "layerType": layer_type,
        "activation": activation_type,
        "weightPath": weight_path,
        "biasPath": bias_path,
        "layer_config" : layer_config
    }

    return layerJson



save_path = f"./saved_models/classifier_v3"
modelJson = {
    "modelName": "classifier_v3",
    "layers": [],
}


index = 0
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers[1:]:
            layerJson = save_layer_file(index, l2, save_path)
            modelJson["layers"].append(layerJson)
            index += 1
    else:
        print(l.name)
        layerJson = save_layer_file(index, l, save_path)
        modelJson["layers"].append(layerJson)
        index += 1

import json

with open(f"{save_path}/model.json", "w") as file:
    file.write(json.dumps(modelJson))
    print("model exported ...")

input_2
conv2d_3
activation_2
conv2d_4
activation_3
conv2d_5
flatten_1
dense_1
model exported ...


# create the numpy dense/softmax layer

In [ ]:
from Layers import Dense
from Activations import Softmax

softmax = Softmax()
fc = Dense("dense", activation=softmax)
fc.load_weights(
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.weights.npy",
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.bias.npy",
)

# test dense/softmax layer

In [ ]:
# the tensorflow layer and the numpy layer on the same input

denseLayer = classifier.get_layer("dense")

input_test = np.ones((32, denseLayer.get_weights()[0].shape[0]))

o_dut = fc(input_test, False).astype(np.float16).ravel()
o_dur = denseLayer(input_test).numpy().astype(np.float16).ravel()

np.savetxt("dut_fc.txt", o_dut)
np.savetxt("dur_fc.txt", o_dur)

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(o_dut == o_dur), f"val test failed. Check dut_fc and dur_fc .txt files for more details."


print("test_passed!")

# create conv2D numpy layer

In [ ]:
from Layers import Conv2D
from Activations.ActivationFactory import activation_from_name

conv2D_ref = classifier.layers[1].get_layer("conv2d")
input_test = np.ones((1, 13, 13, conv2D_ref.weights[0].shape[-2]))


npConv2D = Conv2D(
    strides=conv2D_ref.get_config()["strides"],
    padding=conv2D_ref.get_config()["padding"],
    activation=activation_from_name(conv2D_ref.get_config()["activation"]),
)
npConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/2.Conv2D.conv2d.tanh.weights.npy",
    bias_path="./saved_models/classifier_v2/2.Conv2D.conv2d.tanh.bias.npy",
)


assert np.all(conv2D_ref.weights[0] == npConv2D.weights), "weights are different"
assert np.all(conv2D_ref.weights[1] == npConv2D.bias), "bias are different"
print("weights loaded correctly...")

# test conv2D layer

In [ ]:
o_dut = npConv2D(input_test, training=False).astype(np.float16)
o_dur = conv2D_ref(input_test).numpy().astype(np.float16)

np.savetxt("dut_conv.txt", o_dut.ravel())
np.savetxt("dur_conv.txt", o_dur.ravel())

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(
    o_dut == o_dur
), f"val test failed. Check dut_conv.txt and dur_conv.txt files. "

print("test passed!")

# create depthwise convolution

In [ ]:
from Layers import DephtwiseConv2D

depthwise_conv2d_ref = classifier.layers[1].get_layer("depthwise_conv2d")

npDephtwiseConv2D = DephtwiseConv2D(
    strides=depthwise_conv2d_ref.get_config()["strides"],
    padding=depthwise_conv2d_ref.get_config()["padding"],
    activation=activation_from_name(depthwise_conv2d_ref.get_config()["activation"]),
)
npDephtwiseConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.weights.npy",
    bias_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.bias.npy",
)

assert np.all(
    depthwise_conv2d_ref.weights[0] == npDephtwiseConv2D.weights
), "weights are different"
assert np.all(
    depthwise_conv2d_ref.weights[1] == npDephtwiseConv2D.bias
), "bias are different"
print("weights loaded correctly...")

# test depthwise convolution

In [ ]:
input_test = np.ones((1, 32, 32, depthwise_conv2d_ref.weights[0].shape[-2]))

o_dut = npDephtwiseConv2D(input_test, training=False).astype(np.float16)
o_dur = depthwise_conv2d_ref(input_test).numpy().astype(np.float16)

np.savetxt("dut_dconv.txt", o_dut.ravel())
np.savetxt("dur_dconv.txt", o_dur.ravel())

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(
    o_dut == o_dur
), f"val test failed. Check dut_dconv.txt and dur_dconv.txt files "

print("test passed!")

# create numpy model

In [5]:
import json
from NumpyModel import Model
from Layers import InputLayer, Conv2D, Dense, DephtwiseConv2D, Flatten, ActivationLayer
from Activations.ActivationFactory import activation_from_name
import numpy as np


classifier_model_path = "/home/webphy/Desktop/dnn_processor/neural_network/handmade/saved_models/classifier_v3/"
classifier = tf.keras.models.load_model(
    r"/home/webphy/Desktop/dnn_processor/neural_network/handmade/saved_models/manual_test_4x/model.h5"
)

model = Model()
model.load_model(classifier_model_path)

print(model.layers)

[<Layers.InputLayer.InputLayer object at 0x7f010c0e3e80>, <Layers.Conv2D.Conv2D object at 0x7f0044c21cc0>, <Layers.ActivationLayer.ActivationLayer object at 0x7f0044c21c30>, <Layers.Conv2D.Conv2D object at 0x7f0044369960>, <Layers.ActivationLayer.ActivationLayer object at 0x7f004436a3e0>, <Layers.Conv2D.Conv2D object at 0x7f0044369ea0>, <Layers.Flatten.Flatten object at 0x7f0044369e70>, <Layers.Dense.Dense object at 0x7f004436a0b0>]


# test numpy model

In [6]:
input_test = np.zeros(
    shape=[
        1,
    ]
    + list(classifier.layers[0].get_config()["batch_input_shape"][1:])
)

o_dut_model = classifier(input_test).numpy().astype(np.float16)
o_ref_model = model(input_test, training=False).astype(np.float16)

assert np.all(o_dut_model == o_ref_model), "val test failed. Check values"

print("model test passed...")

2024-02-06 19:37:03.770214: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


AttributeError: 'tuple' object has no attribute 'astype'